### **Root Init**

In [1]:
from pathlib import Path, PurePath
import sys, os, platform

CWD  = Path.cwd().resolve()
ROOT = CWD if (CWD / "src").exists() else CWD.parent
if str(ROOT) not in sys.path: sys.path.append(str(ROOT))

print("ROOT:", ROOT)
print("Python:", platform.python_version())

ROOT: D:\IIT BBS\Job Resources\Business Optima\pdf-agent
Python: 3.11.13


### **New prompts**

In [2]:
PROMPTS = ROOT / "configs" / "prompts"
PROMPTS.mkdir(parents=True, exist_ok=True)

need = {
    "planner.txt": """(paste the PLANNER prompt from above)""",
    "polisher.txt": """(paste the POLISHER prompt from above)""",
}
for name, txt in need.items():
    fp = PROMPTS / name
    if not fp.exists() or fp.stat().st_size == 0:
        fp.write_text(txt.strip() + "\n", encoding="utf-8")
        print("[created]", fp)
    else:
        print("[ok]", fp)

[ok] D:\IIT BBS\Job Resources\Business Optima\pdf-agent\configs\prompts\planner.txt
[ok] D:\IIT BBS\Job Resources\Business Optima\pdf-agent\configs\prompts\polisher.txt


### **Smoke: complex ask**

In [3]:
import uuid, asyncio
from src.agent.orchestrator import Title17Agent

async def run_once(q: str):
    agent = Title17Agent()
    sid = f"sess-{uuid.uuid4().hex[:8]}"
    print("[session]", sid)
    print("[user]", q, "\n")

    final = []
    async for ev in agent.achat_stream(sid, q):
        if ev["type"] == "token":
            print(ev["text"], end="", flush=True)
            final.append(ev["text"])
        elif ev["type"] == "final":
            print("\n\n[FINAL]\n", ev["text"])
            if ev.get("citations"):
                print("\n[CITATIONS]")
                for c in ev["citations"]: print(c)
        elif ev["type"] == "error":
            print("\n[ERROR]", ev["text"])
    return sid, "".join(final)

q = (
 "I need (1) the fair use factors under §107, (2) a short compare/contrast "
 "with §110 exemptions, and (3) a 5-bullet summary of §114 performance rights caveat. "
 "End with page ranges."
)
await run_once(q)

D:\IIT BBS\Job Resources\Business Optima\pdf-agent\src\agent\memory.py:60: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=CFG.ollama_summarizer, temperature=0.2)
D:\IIT BBS\Job Resources\Business Optima\pdf-agent\src\agent\memory.py:61: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  return ConversationSummaryBufferMemory(


[session] sess-79760191
[user] I need (1) the fair use factors under §107, (2) a short compare/contrast with §110 exemptions, and (3) a 5-bullet summary of §114 performance rights caveat. End with page ranges. 

 Answer:
- The four fair use factors are: copying purpose and character, copyri > (2) A performance or display of a work in a performance or display made without authorization under subsection (a)(1); (3) A transmission of a performance or display made without authorization under paragraph (1); and (4) A transmission of a performance or display that would be an indirect transmission of private home viewing under subparagraph (B)). 

Compare/Contrast:
The fair use doctrine allows certain activities that might infringe copyrights if done directly, but protects them if performed indirectly or incidental to another permitted activity. This exemption applies to specific works and conditions, such as educational and non-commercial use.
Summarize (5 bullets):
- A direct performance or

('sess-79760191',
 ' Answer:\n- The four fair use factors are: copying purpose and character, copyri > (2) A performance or display of a work in a performance or display made without authorization under subsection (a)(1); (3) A transmission of a performance or display made without authorization under paragraph (1); and (4) A transmission of a performance or display that would be an indirect transmission of private home viewing under subparagraph (B)). \n\nCompare/Contrast:\nThe fair use doctrine allows certain activities that might infringe copyrights if done directly, but protects them if performed indirectly or incidental to another permitted activity. This exemption applies to specific works and conditions, such as educational and non-commercial use.\nSummarize (5 bullets):\n- A direct performance or display of a work is not considered an infringement if it meets certain conditions, such as public performance, non-commercial use, and three exceptions.')

### **DB check**

In [4]:
import json, sqlite3
from src.agent.config import CFG

con = sqlite3.connect(CFG.sqlite_path); cur = con.cursor()
for table in ["conversations", "messages", "events"]:
    row = cur.execute(f"SELECT COUNT(*) FROM {table}").fetchone()
    print(f"{table}: {row[0]} rows")

print("\n[last 10 events]")
for r in cur.execute("SELECT ts,event,substr(payload,1,160) FROM events ORDER BY id DESC LIMIT 10"):
    print(r)
con.close()


conversations: 2 rows
messages: 4 rows
events: 3 rows

[last 10 events]
(1756035317.220416, 'retrieve_done', '{"session_id": "sess-79760191", "n_hits": 6}')
(1756032824.9794493, 'retrieve_done', '{"session_id": "sess-63dbd528", "n_hits": 6}')
(1756032818.9491704, 'bootstrap', '{"note": "fresh init"}')


### **Out-of-scope test**

In [5]:
await run_once("write a python crawler for reddit and summarize 10 threads")

[session] sess-efe124b8
[user] write a python crawler for reddit and summarize 10 threads 



[FINAL]
 I’m a Title 17 assistant. I can only answer questions that are clearly about U.S. Copyright law (Title 17). Please rephrase your question to reference the relevant sections or concepts.


('sess-efe124b8', '')